In [1]:
import json
from time import sleep
import logging
from tqdm.notebook import tqdm

from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
def execute(query: str, endpoint_url: str = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'):
    """
    https://dbpedia.org/sparql
    https://query.wikidata.org/bigdata/namespace/wdq/sparql
    """
    
    sparql = SPARQLWrapper(endpoint_url)
    # sparql.agent = str(agent_header)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    response = sparql.query().convert()
    return response

In [3]:
qald_test = json.load(open('../data/qald_9_plus_test_dbpedia.json'))
qald_train = json.load(open('../data/qald_9_plus_train_dbpedia.json'))

In [7]:
qald_test['questions'][0]

{'id': '99',
 'question': [{'language': 'en',
   'string': 'What is the time zone of Salt Lake City?'},
  {'language': 'de', 'string': 'In welcher Zeitzone liegt Salt Lake City?'},
  {'language': 'de', 'string': 'Was ist die Zeitzone von Salt Lake City?'},
  {'language': 'ru', 'string': 'Какой часовой пояс в Солт-Лейк-Сити'},
  {'language': 'ru',
   'string': 'В каком часовом поясе расположен Солт-Лейк-Сити?'},
  {'language': 'uk', 'string': 'Який часовий пояс у Солт-Лейк Сіті?'},
  {'language': 'lt', 'string': 'Kokia Solt Leik Sičio laiko zona?'},
  {'language': 'be', 'string': 'Які гадзінны пояс у Солт-Лэйк-Сіці'},
  {'language': 'lt', 'string': 'Kokia laiko juosta yra Solt Leik Sityjes'},
  {'language': 'ba', 'string': 'Ниндей вакыт поясы Солт-Лейк-Ситила'}],
 'query': {'sparql': 'PREFIX res: <http://dbpedia.org/resource/> PREFIX dbp: <http://dbpedia.org/property/> SELECT DISTINCT ?uri WHERE { res:Salt_Lake_City <http://dbpedia.org/ontology/timeZone> ?uri }'},
 'answers': [{'head': 

In [16]:
for q in tqdm(qald_test['questions']):
    if 'query' in q:
        try:
            execute(query=q['query']['sparql'], endpoint_url='https://skynet.coypu.org/dbpedia/')
        except Exception as e:
            with open("test_logs.tsv", "a") as myfile:
                myfile.write(q['id'] + '\t' + q['query']['sparql'].replace("\n", " ") + '\t' + str(e) + '\n')

In [17]:
for q in tqdm(qald_train['questions']):
    if 'query' in q:
        try:
            execute(query=q['query']['sparql'], endpoint_url='https://skynet.coypu.org/dbpedia/')
        except Exception as e:
            with open("train_logs.tsv", "a") as myfile:
                myfile.write(q['id'] + '\t' + q['query']['sparql'].replace("\n", " ") + '\t' + str(e) + '\n')

## Re-run the queries to get the new answers

In [6]:
for q in tqdm(qald_test['questions']):
    q["answers"] = execute(query=q['query']['sparql'], endpoint_url='https://skynet.coypu.org/dbpedia/')

In [7]:
for q in tqdm(qald_train['questions']):
    q["answers"] = execute(query=q['query']['sparql'], endpoint_url='https://skynet.coypu.org/dbpedia/')

In [ ]:
with open("../data/qald_9_plus_test_dbpedia.json", "w") as myfile:
    json.dump(qald_test, myfile, indent=4, ensure_ascii=False)

In [ ]:
with open("../data/qald_9_plus_train_dbpedia.json", "w") as myfile:
    json.dump(qald_train, myfile, indent=4, ensure_ascii=False)